In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.4.2


In [2]:
!pip list | grep scikit-learn

scikit-learn      1.4.2


In [3]:
!python -V

Python 3.11.7


In [1]:
import pickle
import pandas as pd

In [2]:
with open('web-service/lin_reg.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [3]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [6]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [12]:
import numpy as np

np.std(y_pred)

6.227201007193957

In [14]:
results_df = pd.DataFrame.from_dict({'predicted_duration': y_pred})

In [17]:
year = 2023
month = 3

In [18]:
results_df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [20]:
results_df.to_parquet(
    'results.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)

In [21]:
!ls -lh results.parquet

-rw-rw-rw- 1 codespace codespace 66M Jun 14 14:25 results.parquet


In [22]:
!jupyter nbconvert --to script 04-deployment.ipynb

[NbConvertApp] Converting notebook 04-deployment.ipynb to script
[NbConvertApp] Writing 1553 bytes to 04-deployment.py
